In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
RANDOM_SEED = 0xdeadbeef

# Lab 03: Linear Regression

For the first few Tasks, we will work with synthetic univariate data.
We generate $100$ features $x_i \in [-1, 1]$ as `x` and two different
regression targets `y1` and `y2`.

In [ ]:
data_rng = np.random.default_rng(RANDOM_SEED)
n = 100
x = 2 * data_rng.random(n) - 1  # create n points between -1 and 1

# setup synthetic linear data
true_offset = 0.5
true_slope = 1.25
noise = data_rng.normal(loc=0., scale=0.25, size=(n,))

y1 = true_offset + true_slope * x + noise


# setup synthetic non-linear data
y2  = true_offset + np.sin(np.pi * x) + noise

# Task 1 (2 Points): Pearson Correlation

### Task 1a

Plot `x` against the target variable `y1`.

* use `plt.scatter`


Do you think there is a linear relationship between `x` and the target?

In [ ]:
plt.scatter(x, y1)
plt.xlabel("x")
plt.ylabel("y1")
plt.show()

Plot `x` against the target variable `y2`.

Do you think there is a linear relationship between `x` and the target?

In [ ]:
plt.scatter(x, y2)
plt.xlabel("x")
plt.ylabel("y2")
plt.show()

### Task 1b

In class you have seen the formula for the Pearson Correlation:
$\rho(a, b) = \frac{\sum_{i = 1}^{m} (a_i - \bar{a})(b_i - \bar{b})}{\sqrt{\sum_{i=1}^{m} (a_i - \bar{a})^2\sum_{i = 1}^{m}(b_i - \bar{b})^2}} $, where $\bar{a} = \frac{1}{m}\sum_{i=1}^{m} a_i$ and $\bar{b} = \frac{1}{m}\sum_{i=1}^{m} b_i$.

* Compute the Pearson Correlation $\rho$ between `x` and the target `y1`.
* Compute the Pearson Correlation between `x` and `y2`.
* Check that you get the same result as the reference implementation

In [ ]:
def my_pearson(x, y):
  mean_x = np.mean(x)
  mean_y = np.mean(y)

  cent_x = x - mean_x
  cent_y = y - mean_y

  numerator = np.sum(cent_x * cent_y)
  denominator = np.sqrt(np.sum(cent_x * cent_x) * np.sum(cent_y * cent_y))

  return numerator / denominator

print(f"rho(x, y1): {my_pearson(x, y1):.4f}")
print(f"rho(x, y2): {my_pearson(x, y2):.4f}")

In [ ]:
# Refer to the output of this cell to check whether your implementation of rho
# is correct.

from scipy.stats import pearsonr

print(f"rho(x, y1): {pearsonr(x, y1)[0]:.4f}")
print(f"rho(x, y2): {pearsonr(x, y2)[0]:.4f}")

## 📢 **HAND-IN** 📢: Report in Moodle whether you solved this task.

# Task 2 (3 Points): Univariate Linear Regression

### Task 2a

You will now implement Linear Regression with a single variable. In class you have seen that the underlying model is: $y = \theta_0 + \theta_1x$.
You also derived the maximum likelihood estimates for $\theta_0$ and $\theta_1$:

* $\hat{\theta}_1 = \frac{\sum_{i=1}^{m} (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^{m}(x_i - \bar{x})^2}$
* $\hat{\theta}_0 = \bar{y} - \hat{\theta}_1\bar{x}$

In the following cell, implement the `.fit` and `.predict` methods: 
* In the `.predict` method you will have to apply the model to the input `x`
* In the `.fit` method you will have to compute $\hat{\theta}_0$ and $\hat{\theta}_1$.

In [ ]:
class UnivariateLinearRegression:

  def __init__(self):
    self.theta_0: float = 0.
    self.theta_1: float = 0.

  def predict(self, x):
    return self.theta_0 + self.theta_1 * x

  def fit(self, x, y):
    mean_x = np.mean(x)
    mean_y = np.mean(y)

    cent_x = x - mean_x
    cent_y = y - mean_y

    theta_1 = np.sum(cent_x * cent_y) / np.sum(cent_x * cent_x)
    theta_0 = mean_y - theta_1 * mean_x

    self.theta_1 = theta_1
    self.theta_0 = theta_0

    return self

### Task 2b

Fit your linear model to `x` and the target `y1` and plot the resulting residuals.

* Create an instance of the class `UnivariateLinearRegression`
* fit the model using its `.fit` method
* get the predicted values, using `.predict`


In [ ]:
lin_reg_uni = UnivariateLinearRegression()
lin_reg_uni.fit(x, y1)
y_pred = lin_reg_uni.predict(x)

* implement the function `plot_residuals`
* use `plot_residuals` to plot the residuals of your model
* study the residual plot: are the 4 basic assumptions fullfilled? (linearity, normality, independence, equality of variance)

In [ ]:
def plot_residuals(y_pred, y_true):
  plt.scatter(y_pred, y_true - y_pred)
  plt.xlabel("Predicted Value")
  plt.ylabel("Residual")
  plt.title("Residual Plot")
  plt.show()

plot_residuals(y_pred, y1)

* Fit another linear model to `x` and `y2`
* get the predicted values
* plot the residuals
* study the residual plot: are the 4 basic assumptions fullfilled? (linearity, normality, independence, equality of variance)

In [ ]:
plot_residuals(y_pred=UnivariateLinearRegression().fit(x, y2).predict(x), y_true=y2)

## 📢 **HAND-IN** 📢: Report in Moodle whether you solved this task.

# Task 3 (4 Points): Univariate Linear Regression using Stochastic Gradient Descent

### Task 3a

In class you have seen an alternative version to estimate the parameters $\theta_i$ of the linear regression models by using Gradient Descent.

For the univariate linear regression model, the stochastic gradient descent updates look like this:
* $\theta_{0}^{(t+1)} = \theta_{0}^{(t)} - \alpha (\theta_{0}^{(t)} + \theta_{1}^{(t)} x_t - y_t)$
* $\theta_{1}^{(t+1)} = \theta_{1}^{(t)} - \alpha (\theta_{0}^{(t)} + \theta_{1}^{(t)} x_t - y_t) x_t$

Here $\alpha$ is the learning rate, and $(x_t, y_t)$ is the data point sampled
at time $t$.


In the following cell, implement the `.fit` and `.predict` methods: 
* In the `.predict` method you will have to apply the model to the input `x`.
* In the `.fit` method you will have to implement the update equations for
$\theta_0$ and $\theta_1$.

In [ ]:
class SGDUnivariateLinearRegression:

  def __init__(self):
    self.theta_0: float = 0.
    self.theta_1: float = 0.
    self.rng = np.random.default_rng(RANDOM_SEED)

  def predict(self, x):
    # y = theta_0 + theta_1 * x
    return self.theta_0 + self.theta_1 * x

  def fit(self, x, y, n_iter: int = 100, learning_rate: float = 1.0):
    for t in range(n_iter):
      sample_ix = self.rng.integers(0, len(x))

      xt = x[sample_ix]
      yt = y[sample_ix]

      residual = (self.theta_0 + self.theta_1 * xt) - yt

      self.theta_0 -= learning_rate * residual
      self.theta_1 -= learning_rate * residual * xt

    return self

### Task 3b

Run SGD for `x` and the target `y1` and compute the mean squared error (MSE).
The MSE is defined as: $\frac{1}{n}\sum_{i=1}^{n} (\hat{y}_i - y_i)^2$, where
$\hat{y}$ are the model predictions.

* Create an instance of the class `SGDUnivariateLinearRegression`
* fit the model using its `.fit` method
* get the predicted values, using `.predict`
* implement the `mse` function
* compute the MSE of your predictions

In [ ]:
def mse(y_pred, y_true):
  res = y_pred - y_true
  return np.mean(res * res)

In [ ]:
sgd_model = SGDUnivariateLinearRegression()
sgd_model.fit(x, y1)
y_pred = sgd_model.predict(x)

print(mse(y_pred, y1))

### Task 3c

You will now plot the learning curves for different learning rates $\alpha$.
A learning curves shows how a model's performance changes with increasing number of update steps.
In our case we will plot the model's MSE as a function of the number of update
steps `n_iter` for different values of `learning_rate`.

In the following cell we setup most of the scaffold to create this plot. Follow
the instructions in the comments to finish the plots.

In [ ]:
n_iters = [50, 100, 200, 500, 1000, 2000]
learning_rates = [1., .1, .01]

# we plot the MSE achieved by the closed form model as a reference
closed_form = UnivariateLinearRegression()
closed_form.fit(x, y1)
mse_base = mse(y_pred=closed_form.predict(x), y_true=y1)
plt.plot(n_iters, np.ones_like(n_iters) * mse_base, label="closed form", linestyle='--', c='b')

for alpha in learning_rates:
  mses = []
  for n_iter in n_iters:
    model = SGDUnivariateLinearRegression()
    model.fit(x, y1, n_iter=n_iter, learning_rate=alpha)
    mse_ = mse(y_pred=model.predict(x), y_true=y1)
    mses.append(mse_)
  plt.plot(n_iters, mses, label=f"alpha = {alpha:.2f}")

plt.xlabel("n_iter")
plt.ylabel("MSE")
plt.legend()
plt.show()

## 📢 **HAND-IN** 📢: A PDF document containing the following:

* the final plot containing learning curves
* a short (2-3 sentences) interpretation of the curves: why do you think they look the way
they do? can you draw any conclusions?

In case you were not able to arrive at the final plot:

* include screenshots of the code you wrote so we can assign partial credit

**Solutions for Tasks 3 and 4 should be in the same document: you will only upload 1 document with your solutions for both tasks!**


# Task 4 (3 Points): Multivariate Linear Regression

In this task we will apply linear regression to non-synthetic data.
The variable `X` is a `pandas` `Dataframe` containing features and `y` contains
the target. Read through the description to get an idea of the different variables.

In [ ]:
from sklearn.datasets import load_diabetes

data = load_diabetes(as_frame=True)

X = data['data']
y = data['target']
description = data['DESCR']

print(description)

### Task 4a

Implement linear regression using `sklearn`.

* create an instance of the class `sklearn.linear_model.LinearRegression`. Refer to the documentation at: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
* call its `.fit` method
* get the predicted values with `.predict`
* plot the residuals
* study the residual plot: are the 4 basic assumptions fullfilled? (linearity, normality, independence, equality of variance)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X, y)

plot_residuals(y_pred=lin_reg.predict(X), y_true=y)

### Task 4b

The estimated parameters $\theta$ of the linear model can be found in the `.coef_` member variable. The feature names can be found in the `.feature_names_in_` member variable. They are the same as the names of the columns of `X` and should be in the same order.

Using these, answer the following questions:

* Which are the 3 most influential features?
* How do you interpret the sign of the coefficients?
* If you had to exclude 1 feature, which one would you select and why?

In [ ]:
for name, coef in zip(lin_reg.feature_names_in_, lin_reg.coef_):
  print(name, coef, sep='\t')

## 📢 **HAND-IN** 📢: A PDF document containing the following:

* your residual plot
* a short (2-3 sentences) interpretation of the residuals. In particular, your
assessment of the 4 basic assumptions.
* your answers to the questions in Task 4b

**Solutions for Tasks 3 and 4 should be in the same document: you will only upload 1 document with your solutions for both tasks!**
